# Домашнее задание
## Параллельные вычисления

Тема: "Решение задачи линейного программирования при помощи параллельного варианта симплекс-метода"

Выполнили: 
- Александров Алексей Н., ИУ8-114
- Сакулин Даниил Игоревич, ИУ8-115
- Зимин Григорий Сергеевич, ИУ8-112

In [6]:
import logging
from pathlib import Path

from src.simplex.simplex_problem import SimplexProblem, Solution

In [7]:
logging.basicConfig(level=logging.INFO, format='%(message)s')

In [8]:
problem = SimplexProblem(Path("input_LLP.json"))

F = c⋅x -> max,
Ax <= b,
x1,x2, ..., xn >= 0
c^T = [-1 -1],
A =
[[0 8]
 [9 7]],
b^T = [1 1].


In [9]:
solution: Solution = problem.solve()

Процесс решения:
Поиск опорного решения: 
Исходная симплекс-таблица:
+----+--------+--------+--------+
|    |  Si0   |   x1   |   x2   |
+----+--------+--------+--------+
| x3 | 1.0000 | 0.0000 | 8.0000 |
| x4 | 1.0000 | 9.0000 | 7.0000 |
| F  | 0.0000 | 1.0000 | 1.0000 |
+----+--------+--------+--------+
Опорное решение найдено!
x1 = x2 = 0, 
x3 = 1.000, x4 = 1.000
Целевая функция: F = 0.000
Поиск оптимального решения:
Разрешающая строка: x4
Разрешающий столбец: x1
+----+---------+---------+--------+
|    |   Si0   |    x4   |   x2   |
+----+---------+---------+--------+
| x3 |  1.0000 | -0.0000 | 8.0000 |
| x1 |  0.1111 |  0.1111 | 0.7778 |
| F  | -0.1111 | -0.1111 | 0.2222 |
+----+---------+---------+--------+
Разрешающая строка: x3
Разрешающий столбец: x2
+----+---------+---------+---------+
|    |   Si0   |    x4   |    x3   |
+----+---------+---------+---------+
| x2 |  0.1250 | -0.0000 |  0.1250 |
| x1 |  0.0139 |  0.1111 | -0.0972 |
| F  | -0.1389 | -0.1111 | -0.0278 |
+----+--